In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import keras
from keras.engine.topology import Layer

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [3]:
def mape_loss_func(preds, labels):
    mask = labels > 5
    return np.mean(np.fabs(labels[mask]-preds[mask])/labels[mask])

def smape_loss_func(preds, labels):
    mask= labels > 5
    return np.mean(2*np.fabs(labels[mask]-preds[mask])/(np.fabs(labels[mask])+np.fabs(preds[mask])))

def mae_loss_func(preds, labels):
    mask= labels > 5
    return np.fabs((labels[mask]-preds[mask])).mean()

def eliminate_nan(b):
    a = np.array(b)
    c = a[~np.isnan(a)]
    return c

# 制作flow 和 near road

In [17]:
randseed = 25
res = 11
num_dets = 30

v = pd.read_csv('../data/v_20_aggragated.csv')
v = v.rename(columns={'Unnamed: 0': 'id'})
# 以上 flow 制作完毕

dist_mat = pd.read_csv('../data/dist_mat.csv', index_col=0)
id_info = pd.read_csv('../data/id2000.csv', index_col=0)
dist_mat.index = id_info['id2']
dist_mat.columns = id_info['id2']
for i in range(len(dist_mat)):
    for j in range(len(dist_mat)):
        if i==j:
            dist_mat.iloc[i, j] = 0

near_id = pd.DataFrame(np.argsort(np.array(dist_mat.iloc[:num_dets, :num_dets])), index = id_info['id2'][:num_dets], columns = id_info['id2'][:num_dets])
# 以上near_road 制作完毕

In [18]:
#载入最近路段数据，流量数据
# distance = np.array(pd.read_csv('video_link_dis.csv',header = None))
near_road = np.array(near_id)
flow = np.array(v.iloc[:, 1:]) #注意header=0 or None

In [22]:
# 利用滑动窗口的方式，重构数据为(n，最近路段数，输入时间窗，总路段数)的形式
k = 5 # 参数k为需考虑的最近路段数
t_p = 31 # 参数t_p为总时间序列长度（天）
t_input = 12 #参数t_input为输入时间窗(10min颗粒度)
t_pre = 3 #参数t_pre为预测时间窗(10min颗粒度) 颗粒度？
num_links = num_dets #参数num_links为总路段数
flow = flow[:num_links, :]
g = 10
n = 24*60//g


image = []
for i in range(np.shape(near_road)[0]):
    road_id = []
    for j in range(k):
        road_id.append(near_road[i][j])
    image.append(flow[road_id, :])
image1 = np.reshape(image, [-1, k, len(flow[0,:])])
image2 = np.transpose(image1,(1,2,0))
image3 = []
label = []
day = []

for i in range(19, t_p):  # 10 days in total
    for j in range(n-t_input-t_pre):
        image3.append(image2[:, i*n+j:i*n+j+t_input, :][:])
        label.append(flow[:, i*n+j+t_input:i*n+j+t_input+t_pre][:])
        day.append(flow[:, (i-1)*n+j+t_input:(i-1)*n+j+t_input+t_pre][:])

image3 = np.asarray(image3)
label = np.asarray(label)
day =  np.asarray(day)

print(np.shape(image3))
print(np.shape(label))
print(np.shape(day))

#划分前80%数据为训练集，最后20%数据为测试集
prop = 0.3
image_train = image3[:int(np.shape(image3)[0]*prop)]
image_test = image3[int(np.shape(image3)[0]*prop):]
label_train = label[:int(np.shape(label)[0]*prop)]
label_test = label[int(np.shape(label)[0]*prop):]

day_train = day[:int(np.shape(day)[0]*prop)]
day_test = day[int(np.shape(day)[0]*prop):]

(1548, 5, 12, 30)
(1548, 30, 3)
(1548, 30, 3)


In [23]:
# define merge layer
class Merge_Layer(Layer):
    def __init__(self, **kwargs):
        super(Merge_Layer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.para1 = self.add_weight(shape=(input_shape[0][1], input_shape[0][2]),
                                     initializer='uniform', trainable=True,
                                     name='para1')
        self.para2 = self.add_weight(shape=(input_shape[1][1], input_shape[1][2]),
                                     initializer='uniform', trainable=True,
                                     name='para2')
        super(Merge_Layer, self).build(input_shape)

    def call(self, inputs):
        mat1 = inputs[0]
        mat2 = inputs[1]
        output = mat1 * self.para1 + mat2 * self.para2
        # output = mat1 * 0.1 + mat2 * 0.9
        return output

    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [24]:
input_data = keras.Input(shape=(k,t_input,num_links), name='input_data')
input_HA = keras.Input(shape=(num_links, t_pre), name='input_HA')

x = keras.layers.BatchNormalization(input_shape =(k,t_input,num_links))(input_data)

x = keras.layers.Conv2D(
                           filters = 30,
                           kernel_size = 3,
                           strides = 1,
                           padding="SAME",
                           activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)

x = keras.layers.BatchNormalization()(x)

x = keras.layers.Conv2D(
                       filters = 30,
                       kernel_size = 3,
                       strides = 1,
                       padding="SAME",
                       activation='relu')(x)

x = keras.layers.AveragePooling2D(pool_size = (2,2),
                                strides = 1,
                                padding = "SAME",
                                )(x)
x = keras.layers.Flatten()(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.5)(x)

x = keras.layers.Dense(num_links*2*t_pre, activation='relu', name='dense_1')(x)
x = keras.layers.Dense(num_links*t_pre, activation='relu', name='dense_2')(x)

output = keras.layers.Reshape((num_links,t_pre))(x)

output_final = Merge_Layer()([output, input_HA])

# construct model
finish_model = keras.models.Model([input_data,input_HA], [output_final])

finish_model.summary()








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_data (InputLayer)         (None, 5, 12, 30)    0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 5, 12, 30)    120         input_data[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 5, 12, 30)    8130        batch_normalization_1[0][0]      
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 5, 12, 30)    0           conv2d_1[0][0]       

In [25]:
finish_model.compile(optimizer='adam', loss='mean_squared_error')

In [26]:
X_train = image_train
X_HA_train = day_train
label_train = label_train

In [30]:
#模型拟合与评估
finish_model.fit([X_train,X_HA_train], label_train, epochs=350, batch_size=128,
validation_data=([image_test,day_test], label_test))
# finish_model.evaluate(image_test, label_test)

Train on 464 samples, validate on 1084 samples
Epoch 1/100
464/464 [==============================] - 0s 180us/step - loss: 291.0429 - val_loss: 401.1940
Epoch 2/100
464/464 [==============================] - 0s 166us/step - loss: 287.8879 - val_loss: 397.0823
Epoch 3/100
464/464 [==============================] - 0s 159us/step - loss: 284.6632 - val_loss: 397.8468
Epoch 4/100
464/464 [==============================] - 0s 150us/step - loss: 282.3822 - val_loss: 395.4634
Epoch 5/100
464/464 [==============================] - 0s 159us/step - loss: 279.3455 - val_loss: 392.8933
Epoch 6/100
464/464 [==============================] - 0s 155us/step - loss: 276.6873 - val_loss: 388.2605
Epoch 7/100
464/464 [==============================] - 0s 148us/step - loss: 276.0499 - val_loss: 393.0555
Epoch 8/100
464/464 [==============================] - 0s 150us/step - loss: 270.7546 - val_loss: 384.2551
Epoch 9/100
464/464 [==============================] - 0s 155us/step - loss: 268.5143 - val_loss:

464/464 [==============================] - 0s 144us/step - loss: 153.0164 - val_loss: 268.3155
Epoch 77/100
464/464 [==============================] - 0s 144us/step - loss: 151.4842 - val_loss: 265.8886
Epoch 78/100
464/464 [==============================] - 0s 142us/step - loss: 149.4084 - val_loss: 265.0862
Epoch 79/100
464/464 [==============================] - 0s 140us/step - loss: 148.1926 - val_loss: 264.4480
Epoch 80/100
464/464 [==============================] - 0s 142us/step - loss: 145.9446 - val_loss: 262.7847
Epoch 81/100
464/464 [==============================] - 0s 140us/step - loss: 146.1090 - val_loss: 262.8665
Epoch 82/100
464/464 [==============================] - 0s 144us/step - loss: 144.6720 - val_loss: 261.2390
Epoch 83/100
464/464 [==============================] - 0s 146us/step - loss: 144.3645 - val_loss: 259.9115
Epoch 84/100
464/464 [==============================] - 0s 191us/step - loss: 143.7077 - val_loss: 262.9430
Epoch 85/100
464/464 [===================

In [31]:
#模型预测
model_pre = finish_model.predict([image_test,day_test])

In [32]:
#计算各项误差指标

mape_mean = mape_loss_func(model_pre, label_test)
smape_mean = smape_loss_func(model_pre, label_test)
mae_mean = mae_loss_func(model_pre, label_test)

print('mape = ' + str(mape_mean) + '\n' + 'smape = ' + str(smape_mean) + '\n' + 'mae = ' + str(mae_mean))

mape = 0.21521104344971367
smape = 0.2360158251183316
mae = 11.709483633314415


In [33]:
#模型保存
finish_model.save_weights('../model/source_base.h5')

In [34]:
#计算每条路段的误差
mape_list = []
for i in range(num_links):
    a1 = mape_loss_func(model_pre[:,i,:], label_test[:,i,:])
    mape_list.append(a1)
    print(str(i+1)+'th link')
    print(a1)

1th link
0.2319396189000425
2th link
0.18734180501204206
3th link
0.26595153238498426
4th link
0.20522465484080915
5th link
0.15505751159335412
6th link
0.5019992480669413
7th link
0.12813163179812861
8th link
0.2260138115716322
9th link
0.18740360287651478
10th link
0.16601372022969269
11th link
0.17290482777199137
12th link
0.19281218938116698
13th link
0.15031497868922367
14th link
0.1892829247854927
15th link
0.24455300179016304
16th link
0.19068467999480912
17th link
0.11461669479137482
18th link
0.30357015538496795
19th link
0.20882560335418207
20th link
0.18037636506757057
21th link
0.16977098246137
22th link
0.229986257109525
23th link
0.33510372356746937
24th link
0.25765544605252655
25th link
0.16739174880016558
26th link
0.16498515029338204
27th link
0.23503522759556267
28th link
0.2712577944263715
29th link
0.15626722579710306
30th link
0.28098749162994135


In [35]:
mape_pd = pd.Series(mape_list)

In [36]:
mape_pd.sort_values()

16    0.114617
6     0.128132
12    0.150315
4     0.155058
28    0.156267
25    0.164985
9     0.166014
24    0.167392
20    0.169771
10    0.172905
19    0.180376
1     0.187342
8     0.187404
13    0.189283
15    0.190685
11    0.192812
3     0.205225
18    0.208826
7     0.226014
21    0.229986
0     0.231940
26    0.235035
14    0.244553
23    0.257655
2     0.265952
27    0.271258
29    0.280987
17    0.303570
22    0.335104
5     0.501999
dtype: float64